### This script allows to get better control over tasks in earth engine

* Purpose of script: allows the user to list tasks and cancel all pending tasks
* Author: Rutger Hofste
* Kernel used: python27
* Date created: 20170913

In [1]:
import time
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
print(dateString,timeString)

('Y2017M12D19', 'UTC 10:19')


# Settings

In [2]:
MAXTASKS = 160
CANCELTASKS = 1

In [3]:
import pandas as pd
import ee
from retrying import retry
import datetime
import random

In [4]:
ee.Initialize()

# Functions

In [5]:
def get_tasks():
    return ee.batch.Task.list()

def cancel_task(task):
    print task
    random_time = random.random()
    time.sleep(0.5+random_time*0.5)
    if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY') :
        print 'canceling %s' % task
        task.cancel()
        
        
@retry(wait_exponential_multiplier=10000, wait_exponential_max=100000)
def checkStatus(task):
    return ee.batch.Task.status(task)
           
def get_details(taskList,MAXTASKS):
    df = pd.DataFrame()
    for i in range(0,min(len(taskList),MAXTASKS)):
        dictNew = checkStatus(taskList[i])
        dfNew = pd.DataFrame(dictNew, index=[i])
        try:
            dfNew["calctime(min)"] = (dfNew["update_timestamp_ms"]-dfNew["start_timestamp_ms"])/(1000*60)
            dfNew["queuetime(min)"] = (dfNew["start_timestamp_ms"]-dfNew["creation_timestamp_ms"])/(1000*60)
            dfNew["runtime(min)"]= dfNew["queuetime(min)"]+dfNew["calctime(min)"]
            dfNew["start_timestamp_UTC"] = datetime.datetime.fromtimestamp(dfNew["start_timestamp_ms"]/1000).strftime('%H:%M:%S')
        except:
            pass
        df = df.append(dfNew)
        print i
    return df
    


In [6]:
taskList = get_tasks()


In [7]:
detailedTasks = get_details(taskList,MAXTASKS)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159


In [8]:
detailedTasks

,calctime(min),creation_timestamp_ms,description,id,progress,queuetime(min),runtime(min),start_timestamp_UTC,start_timestamp_ms,state,task_type,update_timestamp_ms
0,NaN,1513678626884,reduced_global_historical_combined_short_mean_...,RVBCROELLIV7CRWK7UGSFWFE,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1513678626884
1,NaN,1513678624621,reduced_global_historical_combined_short_mean_...,BW2TZ3ZGKRKNDQ3FUZDRMN7P,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1513678624621
2,NaN,1513678622361,reduced_global_historical_combined_short_mean_...,SURH5R5RUIF3UAIZOFGD3GXL,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1513678622361
3,NaN,1513678619067,reduced_global_historical_combined_short_mean_...,QCLRMR4V7L2DBP6SC653ZCGO,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1513678619067
4,NaN,1513678617230,reduced_global_historical_combined_short_mean_...,GBCEKPRN5PEEZY2UFKWFN2R5,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1513678617230
5,NaN,1513678614952,reduced_global_historical_combined_short_mean_...,HZZUYAS6T37FVG35DIHGLXUY,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1513678614952
6,NaN,1513678612658,reduced_global_historical_combined_short_mean_...,TZYNXMFQ5RLTMRPMH23AIIE4,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1513678612658
7,NaN,1513678609432,reduced_global_historical_combined_short_mean_...,TWPPPDIS6GDDRXSUNN4M42ML,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1513678609432
8,NaN,1513678607194,reduced_global_historical_combined_short_mean_...,BIQLGT7JBYWTHHD5KDSKWYC7,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1513678607194
9,NaN,1513678604965,reduced_global_historical_combined_long_mean_Q...,AU3AWWOWSLJMZMLYERK7V7TC,NaN,NaN,NaN,NaN,NaN,READY,EXPORT_IMAGE,1513678604965


# DANGER ZONE

In [9]:
if CANCELTASKS == 1:
    pendingTasks = [task for task in taskList if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY')]
    for task in pendingTasks:
        cancel_task(task)

<Task EXPORT_IMAGE: reduced_global_historical_combined_short_mean_Q_V04 (READY)>
canceling <Task EXPORT_IMAGE: reduced_global_historical_combined_short_mean_Q_V04 (READY)>
<Task EXPORT_IMAGE: reduced_global_historical_combined_short_mean_Q_V04 (READY)>
canceling <Task EXPORT_IMAGE: reduced_global_historical_combined_short_mean_Q_V04 (READY)>
<Task EXPORT_IMAGE: reduced_global_historical_combined_short_mean_Q_V04 (READY)>
canceling <Task EXPORT_IMAGE: reduced_global_historical_combined_short_mean_Q_V04 (READY)>
<Task EXPORT_IMAGE: reduced_global_historical_combined_short_mean_Q_V04 (READY)>
canceling <Task EXPORT_IMAGE: reduced_global_historical_combined_short_mean_Q_V04 (READY)>
<Task EXPORT_IMAGE: reduced_global_historical_combined_short_mean_Q_V04 (READY)>
canceling <Task EXPORT_IMAGE: reduced_global_historical_combined_short_mean_Q_V04 (READY)>
<Task EXPORT_IMAGE: reduced_global_historical_combined_short_mean_Q_V04 (READY)>
canceling <Task EXPORT_IMAGE: reduced_global_historical_com